## Visualize all tracks in file '2017-05-11_00h_tracks.csv' :

In [199]:
import pandas as pd
from pandas import DataFrame
import numpy as np
import os
import csv

num_TOs = 1500

# Loading original tracks data:
cols = ['ID','Time','x', 'y','Speed','Type','Estimated']
data = pd.read_csv('2017-05-11_00h_tracks.csv',index_col=False, header=None,sep=';',names=cols )
data = data.drop(data.index[0])   # drops the first row
df = data[(data['Type'] == 2)]  # only selecting TO of type 2 (car)


# creating new data frame for plotting trajectories: 
df = df[['ID', 'Time','x', 'y']]

print(df.head())

# path to to put saved file:
path = r'C:\Users\Katarina Tran\Desktop\CAS\Project_course_math_stat_modelling\2017_05_11_L'

# the file is currently named 'selected_path.csv':
df.to_csv(os.path.join(path, r'xy_L_2017_05_11_filtered.csv'), index=False)

print('Selected paths saved to csv !')

               ID                              Time        x        y
133578  140016899  2017-05-11 07:41:54.445000+02:00 -6.34335  12.9815
133579  140016899  2017-05-11 07:41:54.699000+02:00 -6.11119  11.9793
133580  140016899  2017-05-11 07:41:54.951000+02:00 -5.87181  11.1854
133581  140016899  2017-05-11 07:41:55.204000+02:00 -5.57613  10.2882
133582  140016899  2017-05-11 07:41:55.457000+02:00 -5.05073  9.39386
Selected paths saved to csv !


## Conversion to lat-long coord:

In [203]:
import pandas as pd
import numpy as np
from pandas import DataFrame
import os
import csv

""" Converts data x-y coordinates into latitudinal and longitudinal positions.
 Specify below the path where the csv file is to be saved. Requires Pandas and Numpy package."""

########################################################################################################################
# lat <=> y, long <=> x
ref_position = np.array([59.916559, 10.734520])
lat = 59.916687
lon = 10.734836
position = [lat, lon]

diff_lat = position[0] - ref_position[0]
diff_lon = position[1] - ref_position[1]

diff_x = 23.41
diff_y = -20

const_lat = np.round(diff_lat/diff_y, 6)
const_lon = np.round(diff_lon/diff_x, 6)

theta = np.pi + np.pi/2

########################################################################################################################

cols = ['ID','Time','x', 'y']

# Add the csv file to be converted here:
#df = pd.read_csv(r'C:\Users\Katarina Tran\Desktop\CAS\Project_course_math_stat_modelling\2017_05_11_L\xy_R_2017_05_11_filtered.csv', index_col=False, header=None, sep=',', names= cols)
df = pd.read_csv(r'C:\Users\Katarina Tran\Desktop\CAS\Project_course_math_stat_modelling\2017_05_11_L\xy_L_2017_05_11_filtered.csv')

df = df.drop(df.index[0])   # drops the first row

df['lat'] = df['y'].astype(float)
df['long'] = df['x'].astype(float)
df['x'] = df['long'].astype(float)
df['y'] = df['lat'].astype(float)

# rotate x-y-coordinates by pi/2:
#df['x_new'] = df['x'].multiply(np.cos(theta)) - df['y'].multiply(np.sin(theta))
#df['y_new'] = df['x'].multiply(np.sin(theta)) + df['y'].multiply(np.cos(theta))

df['const_lat'] = const_lat
df['const_long'] = const_lon
df['ref_lat'] = ref_position[0]
df['ref_long'] = ref_position[1]

df['lat'] = df['y'].multiply(df['const_lat'])
df['lat'] = df['lat'].add(df['ref_lat'])

df['long'] = df['x'].multiply(df['const_long'])
df['long'] = df['long'].add(df['ref_long'])

#df = df.drop(['const_lat', 'const_long', 'ref_lat', 'ref_long', 'x', 'y', 'x_new', 'y_new'], axis=1)
df = df.drop(['const_lat', 'const_long', 'ref_lat', 'ref_long', 'x', 'y'], axis=1)

# rounding positions coordinates to 6 decimals:
df = df.round({'lat': 6, 'long': 6})

# path to to put saved file:
path = r'C:\Users\Katarina Tran\Desktop\CAS\Project_course_math_stat_modelling\2017_05_11_L'

# the file is currently named 'LatLongData.csv':
df.to_csv(os.path.join(path, r'LatLongData_2017_05_11_L.csv'), index=False)

print('conversion completed!')
df_converted = df


conversion completed!


## Creating geojson file:

In [204]:
import pandas as pd
import numpy as np
import json
import csv


''' This script converts the a data frame with columns: 'ID', 'lat', long' into geojson file for plotting '''

########################################################################################################################
'''
# example data:    note: the 'lat' and 'long' must not be of typ numpy.int64 !
Data = {'ID': [1, 1, 1, 1, 2, 2, 2, 2, 2, 2], 'lat':  [10.734221, 10.734474, 10.734661, 10.734940,  10.734114,
                                                       10.734527, 10.734436, 10.734168, 10.733894, 10.733749],
        'long': [59.916174, 59.916397, 59.916545, 59.916757,  59.916206, 59.916521, 59.916615, 59.916679, 59.916765,
                 59.916798]
        }
df = DataFrame(Data, columns=['ID', 'lat', 'long'])

'''
########################################################################################################################
cols = ['ID','Time','lat', 'long']
# Add the csv file to be converted here:
df = pd.read_csv(r'C:\Users\Katarina Tran\Desktop\CAS\Project_course_math_stat_modelling\2017_05_11_L\LatLongData_2017_05_11_L.csv', index_col=False, header=None, sep=',', names=cols)
df = df.drop(df.index[0])   # drops the first row
del df['Time']
#df = df.loc[:, ['ID', 'lat', 'long']]


# choose number of traffic objects to plot:
num_TO = 900
TO_unique = df.ID.unique()
batch = TO_unique[0:num_TO]
df = df[df['ID'].isin(batch)]

df['lat'] = df['lat'].astype(float)
df['long'] = df['long'].astype(float)
def df_to_geojson(df):
    # extracting unique TO IDs:
    TO_unique = df.ID.unique()
    MultiLineString = []
    for object in TO_unique:
        # converts the the data frame to an array  [[ , ],[ , ],...[ , ]] :
        string = df.loc[df['ID'] == object].values
        # deleting first element in the array (corresponding to the ID value):
        string = np.delete(string, np.s_[0], 1)
        num_rows = len(string)
        object_string = []
        for row in range(num_rows):
            string[row][0] = np.round(string[row][0], 6)
            string[row][1] = np.round(string[row][1], 6)
            object_string.append([string[row][1], string[row][0]])
        MultiLineString.append(object_string)

    geojson_dict = {'type': 'FeatureCollection', 'features': [{'type': 'Feature', 'properties': {}, 'geometry':
        {'type':'MultiLineString', 'coordinates': MultiLineString}}]}

    return geojson_dict

geojson_dict = df_to_geojson(df)

geojson_str = json.dumps(geojson_dict, indent=2)

# save the geojson result to a file
output_filename = 'TO_paths_2017_05_11_L.geojson'


#with open(output_filename, 'wb') as output_file:
with open(output_filename, 'w') as output_file:
    output_file.write('{}'.format(geojson_str))

print('geojson created!')

geojson created!


## Code for separating left turn: 

In [205]:
import pandas as pd
from pandas import DataFrame
import numpy as np
import os
import csv


theta = -0.523   #radians

theta2 = 0.785   

# Loading original tracks data:
cols = ['ID','Time','x', 'y','Speed','Type','Estimated']
#cols = ['ID','Time','x', 'y']
data = pd.read_csv('2017-05-11_00h_tracks.csv',index_col=False, header=None,sep=';',names=cols )
data = data.drop(data.index[0])   # drops the first row
df1 = data[(data['Type'] == 2)]  # only selecting TO of type 2 (car)

# rotate x-y-coordinates by theta to align the road with the x-y axis in the map:
df1['x_rot'] = df1['x'].multiply(np.cos(theta)) - df1['y'].multiply(np.sin(theta))
df1['y_rot'] = df1['x'].multiply(np.sin(theta)) + df1['y'].multiply(np.cos(theta))

# rotate x-y-coordinates by theta to align the road with the x-y axis in the map:
df1['x_rot2'] = df1['x'].multiply(np.cos(theta2)) - df1['y'].multiply(np.sin(theta2))
df1['y_rot2'] = df1['x'].multiply(np.sin(theta2)) + df1['y'].multiply(np.cos(theta2))



# creating new data frame for plotting trajectories: 
df = pd.DataFrame(columns=['ID', 'Time', 'x', 'y'])

grouped = df1.groupby(['ID'])   # selecting only TO ID:s that fullfill x-y coord. condition for right turn
for ID,group in grouped:

    df_temp = pd.DataFrame({
                            'x_cond1':[not(group['x'] > 0).any()]
                            ,'x_cond2':[(group['x_rot'] > -3).any()]
                            ,'x_cond3':[(group['x'] < -23).any()]  
                            ,'x_cond4':[not(group['x_rot2'] > 3).any()]  
                            ,'x_cond5':[not(group['x_rot2'] > 4).any()]
                            ,'x_cond6':[(group['x_rot2'] > -3).any()]
                            ,'y_cond1':[not(group['y_rot'] < -14).any()]
                            ,'y_cond2':(group['y_rot'] > 2).any()
                            ,'y_cond3':[(group['y'] < -10).any()] 
                            ,'y_cond4':[(group['y_rot'] < -3.5).any()]
                            ,'y_cond5':[(group['y_rot'] < -3.5).any()]
                           })
    
    if df_temp.all(axis=None):   # if boundary conditions for x and y are fullfilled
        df = df.append(group[['ID','Time','x','y']])
        

#df = df.rename(columns={"x_rot2": "x", "y_rot2": "y"})
#df = df.iloc[:,[0,1,3,5]]

# path to to put saved file:
path = r'C:\Users\Katarina Tran\Desktop\CAS\Project_course_math_stat_modelling\2017_05_11_L'

# the file is currently named 'selected_path.csv':
df.to_csv(os.path.join(path, r'xy_L_2017_05_11_filtered.csv'), index=False)

print('Selected paths saved to csv !')

C:\Users\Katarina Tran\Anaconda3\lib\site-packages\ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Katarina Tran\Anaconda3\lib\site-packages\ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Katarina Tran\Anaconda3\lib\site-packages\ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/sta

Selected paths saved to csv !


## Code for separating right turn :

In [50]:
import pandas as pd
from pandas import DataFrame
import numpy as np
import os
import csv

#num_TOs = 1500
x_left_bound = 5
x_right_bound = 33
y_upper_bound = -30
y_lower_bound = 2

x_condition_2 = 22
y_condition_2 = -15
theta = 0.523   #radians


# Loading original tracks data:
cols = ['ID','Time','x', 'y','Speed','Type','Estimated']
data = pd.read_csv('2017-05-11_00h_tracks.csv',index_col=False, header=None,sep=';',names=cols )
data = data.drop(data.index[0])   # drops the first row
df1 = data[(data['Type'] == 2)]  # only selecting TO of type 2 (car)

# rotate x-y-coordinates by theta to align the road with the x-y axis in the map:
df1['x_rot'] = df1['x'].multiply(np.cos(theta)) - df1['y'].multiply(np.sin(theta))
df1['y_rot'] = df1['x'].multiply(np.sin(theta)) + df1['y'].multiply(np.cos(theta))


# creating new data frame for plotting trajectories: 
df = pd.DataFrame(columns=['ID', 'Time', 'x', 'y'])
print('test')
grouped = df1.groupby(['ID'])   # selecting only TO ID:s that fullfill x-y coord. condition for right turn
for ID,group in grouped:

    df_temp = pd.DataFrame({#'x_cond':group['x'].between(x_left_bound, x_right_bound),
                            #'y_cond':group['y'].between(y_upper_bound, y_lower_bound),
                            'x_cond1':[not(group['x'] > 0).any()]
                            ,'x_cond2':[not(group['x_rot'] > 2).any()]
                            ,'x_cond3':[not(group['x_rot'] < -15).any()]  
                            #,'y_cond1':[not(group['y_rot'] < -4).any()]
                            #,'y_cond2':(group['y_rot'] > 1).any()
                            ,'x_cond3':[(group['x_rot'] <= -22).any()] 
                           # ,'x_cond4':[not(group['x_rot'] > -7).any()]
                            #,'x_cond5':[not(group['x'] >-10).any()]
                            #,'y_cond3':not(group['y_rot'] > 13).any()
                            
                           })
    
    if df_temp.all(axis=None):   # if boundary conditions for x and y are fullfilled
        df = df.append(group[['ID','Time','x','y','Type']])    
#df = df.rename(columns={"x_rot": "x", "y_rot": "y"})
#df = df.iloc[:,[0,1,3,5]]
print(df.head())
# path to to put saved file:
path = r'C:\Users\Katarina Tran\Desktop\CAS\Project_course_math_stat_modelling\2017_05_11_L'

# the file is currently named 'selected_path.csv':
df.to_csv(os.path.join(path, r'xy_L_2017_05_11_filtered2.csv'), index=False)

print('Selected paths saved to csv !')

print(df1.head())

C:\Users\Katarina Tran\Anaconda3\lib\site-packages\ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Katarina Tran\Anaconda3\lib\site-packages\ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


test


KeyboardInterrupt: 

## Checking if time matches between state vector file and xy-file , seems like state vector is ~ 6.5 seconds ahead:

In [184]:
import pandas as pd
import numpy as np
import json
import csv
from datetime import datetime
from datetime import timedelta
import os

# Loading original tracks data:
data = pd.read_csv(r'C:\Users\Katarina Tran\Desktop\CAS\Project_course_math_stat_modelling\2017_05_11_L\xy_L_2017_05_11_filtered.csv')

data = data.drop(data.index[0])   # drops the first row


df1 = data[['ID','Time']]

# Loading data with occupancy grid:
data2 = pd.read_csv(r'C:\Users\Katarina Tran\Desktop\CAS\Project_course_math_stat_modelling\2017_05_11_L\result_2017_05_11_L.csv')

data2['ID'] = data2['Object id']

del data2['Object id']
del data2['State time']


# choosing ID:s in df2 that also exists in df1:
df2 = data2.loc[data2['ID'].isin(df1['ID'])]
df2['ID'] = df2['ID'].astype('int32')
#print(df2.dtypes)
# choosing ID:s in df1 that is also existing in df2:
df1 = df1.loc[df1['ID'].isin(df2['ID'])]
df2 = df2[['ID', 'Time id']]
print(df1.head())


temp = df2.loc[df2['ID']==140016900]

# calculating state time for data containing occupancy grid:
temp['Time id'] = temp['Time id'].astype(float)
start_time = pd.to_datetime('2017-05-11 00:00:07.560') 
temp['State time'] = start_time + timedelta(seconds=0.3)*temp['Time id'] - timedelta(seconds= 6.5)

print('\n')
print('new')
print(temp.head())



          ID                              Time
1  140016900  2017-05-11 07:41:56.871000+02:00
2  140016900  2017-05-11 07:41:57.123000+02:00
3  140016900  2017-05-11 07:41:57.376000+02:00
4  140016900  2017-05-11 07:41:57.629000+02:00
5  140016900  2017-05-11 07:41:57.880000+02:00


new
              ID  Time id              State time
32265  140016900  92386.0 2017-05-11 07:41:56.860
32266  140016900  92387.0 2017-05-11 07:41:57.160
32267  140016900  92388.0 2017-05-11 07:41:57.460
32268  140016900  92389.0 2017-05-11 07:41:57.760
32269  140016900  92390.0 2017-05-11 07:41:58.060


C:\Users\Katarina Tran\Anaconda3\lib\site-packages\ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Katarina Tran\Anaconda3\lib\site-packages\ipykernel_launcher.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Katarina Tran\Anaconda3\lib\site-packages\ipykernel_launcher.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/sta

## Selecting state vector (LEFT turn) which corresponds in time with xy-file cut in approximately same position :

In [193]:

import pandas as pd
from pandas import DataFrame
import numpy as np
from datetime import datetime
from datetime import timedelta
import os

minimum_sequence = 15  # sequence of xy-trajectories for map plotting 
# num_TOs = 1500

# Loading file which contains x-y coord for right turn paths:
cols = ['ID','Time','x','y']
data1 = pd.read_csv(r'C:\Users\Katarina Tran\Desktop\CAS\Project_course_math_stat_modelling\2017_05_11_L\xy_L_2017_05_11_filtered.csv',index_col=False, header=None,sep=',',names=cols )
data1 = data1.drop(data1.index[0])   # drops the first row
#df1 = data1[(data1['Type'] == 2)]  # only selecting TO of type 2 (car), allready filltered out above
df1 = data1

df1['Time'] = pd.to_datetime(df1['Time']) 
df1['Time'] = df1['Time'].astype('datetime64[ns]')
df1['ID'] = df1['ID'].astype('int32')
df1['ID'] = pd.to_numeric(df1['ID'])
df1['x_numeric'] = pd.to_numeric(df1['x'])

# deleting xy-rows which has x-coord. above 18:
indexRows = df1[ df1['x_numeric'] >= 18].index
df1.drop(indexRows , inplace=True)
del df1['x_numeric']
# Loading data with occupancy grid:
data2 = pd.read_csv('result_2017_05_11_L.csv')
data2['ID'] = data2['Object id']
del data2['Object id']
del data2['State time']


# choosing ID:s in df2 that also exists in df1:
df2 = data2.loc[data2['ID'].isin(df1['ID'])]
df2['ID'] = df2['ID'].astype(float)
#print(df2.dtypes)
# choosing ID:s in df1 that is also existing in df2:
df1 = df1.loc[df1['ID'].isin(df2['ID'])]



# calculating state time for data containing occupancy grid:
df2['Time id'] = df2['Time id'].astype(float)
start_time = pd.to_datetime('2017-05-11 00:00:07.560') 
df2['State time'] = start_time + timedelta(seconds=0.3)*df2['Time id'] - timedelta(seconds=6.5)

df_state = df2

# path to to put saved file:
path = r'C:\Users\Katarina Tran\Desktop\CAS\Project_course_math_stat_modelling\2017_05_11_L'

# the file is currently named 'xy_right_time_matching.csv':
df.to_csv(os.path.join(path, r'xy_L_2017_05_11_filtered.csv'), index=False)

# the file is currently named 'state_vector_right_turn.csv':
df_state.to_csv(os.path.join(path, r'state_vect_L_2017_05_11.csv'), index=False)

print('Selected paths saved to csv !')


ValueError: Given date string not likely a datetime.